In [2]:
import pandas as pd
import altair as alt
import numpy as np

In [3]:
flights = pd.read_json("./Project2/flights_missing.json")


In [4]:
# flights = (flights
    # .replace(-999, np.nan)
    # .replace("1500+", 1500)
    # .replace("Febuary", "February")
# )
# flights["num_of_delays_carrier"] = flights["num_of_delays_carrier"].astype(int)

# With a lambda Function
flights = (flights
    .replace(-999, np.nan)
    .replace("1500+", "1500")
    .assign(
        num_of_delays_carrier = lambda x: x.num_of_delays_carrier.astype(int)
    )
)


flights = (flights.assign(
    severe = flights.num_of_delays_weather,
    mild_late = .3 * flights.num_of_delays_late_aircraft,
    mild_nas = np.where(flights.month.isin(["April", "May", "June", "July", "August"]),
        .4*flights.num_of_delays_nas,
        .65*flights.num_of_delays_nas),
    total_weather = lambda x: x.severe + x.mild_late + x.mild_nas)
)

flights

,airport_code,airport_name,month,year,num_of_flights_total,num_of_delays_carrier,num_of_delays_late_aircraft,num_of_delays_nas,num_of_delays_security,num_of_delays_weather,...,minutes_delayed_carrier,minutes_delayed_late_aircraft,minutes_delayed_nas,minutes_delayed_security,minutes_delayed_weather,minutes_delayed_total,severe,mild_late,mild_nas,total_weather
0,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",January,2005.0,35048,1500,NaN,4598,10,448,...,116423.0,104415,207467.0,297,36931,465533,448,NaN,2988.70,NaN
1,DEN,"Denver, CO: Denver International",January,2005.0,12687,1041,928.0,935,11,233,...,53537.0,70301,36817.0,363,21779,182797,233,278.4,607.75,1119.15
2,IAD,,January,2005.0,12381,414,1058.0,895,4,61,...,NaN,70919,35660.0,208,4497,134881,61,317.4,581.75,960.15
3,ORD,"Chicago, IL: Chicago O'Hare International",January,2005.0,28194,1197,2255.0,5415,5,306,...,88691.0,160811,364382.0,151,24859,638894,306,676.5,3519.75,4502.25
4,SAN,"San Diego, CA: San Diego International",January,2005.0,7283,572,680.0,638,7,56,...,27436.0,38445,21127.0,218,4326,91552,56,204.0,414.70,674.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919,IAD,"Washington, DC: Washington Dulles International",December,2015.0,2799,182,183.0,61,0,17,...,NaN,15438,2826.0,0,1825,31164,17,54.9,39.65,111.55
920,ORD,"Chicago, IL: Chicago O'Hare International",December,2015.0,25568,923,1755.0,1364,11,180,...,80962.0,132055,72045.0,435,22459,307956,180,526.5,886.60,1593.10
921,SAN,"San Diego, CA: San Diego International",n/a,2015.0,6231,480,606.0,256,5,37,...,25402.0,35796,9038.0,161,2742,73139,37,181.8,166.40,385.20
922,SFO,"San Francisco, CA: San Francisco International",December,2015.0,13833,757,1180.0,2372,9,147,...,55283.0,96703,193525.0,285,13788,359584,147,354.0,1541.80,2042.80


In [5]:
# Group by function with filtered rows
# worst_delays = flights.groupby("airport_code").sum().reset_index().filter(["airport_code", "num_of_flights_total", "num_of_delays_total"])

worst_delays2 = (flights
    .groupby("airport_code")
    .agg(
        sum_flights_total = ("num_of_flights_total", np.sum),
        sum_flights_delayed = ("num_of_delays_total", np.sum)
    )
    .assign(
         prop_delay = lambda x: x["sum_flights_delay"] / x.sum_flights_total
    )
)

KeyError: 'sum_flights_delay'

In [ ]:
delays_carrier_mean = flights[["num_of_delays_carrier", "airport_code"]].groupby("airport_code").mean().reset_index()
worst_delays3 = (flights["num_of_delays_carrier", "airport_code"]
    .fillna(value=delays_carrier_mean)
    )

In [3]:
flights.query('month == "n/a"').count()


airport_code                     27
airport_name                     27
month                            27
year                             27
num_of_flights_total             27
num_of_delays_carrier            27
num_of_delays_late_aircraft      27
num_of_delays_nas                27
num_of_delays_security           27
num_of_delays_weather            27
num_of_delays_total              27
minutes_delayed_carrier          26
minutes_delayed_late_aircraft    27
minutes_delayed_nas              27
minutes_delayed_security         27
minutes_delayed_weather          27
minutes_delayed_total            27
dtype: int64

In [4]:
flights.count()

airport_code                     924
airport_name                     924
month                            924
year                             901
num_of_flights_total             924
num_of_delays_carrier            924
num_of_delays_late_aircraft      924
num_of_delays_nas                924
num_of_delays_security           924
num_of_delays_weather            924
num_of_delays_total              924
minutes_delayed_carrier          872
minutes_delayed_late_aircraft    924
minutes_delayed_nas              893
minutes_delayed_security         924
minutes_delayed_weather          924
minutes_delayed_total            924
dtype: int64